# Deep Residual Learning for Image Recognition

**Notebook author: Shuang HOU**

At the end of 2015, Microsoft Research Asia released a paper titled ["Deep Residual Learning for Image Recognition"](https://openaccess.thecvf.com/content_cvpr_2016/papers/He_Deep_Residual_Learning_CVPR_2016_paper.pdf), authored by Kaiming He, Xiangyu Zhang, Shaoqing Ren and Jian Sun. The paper achieved state-of-the-art results in Image classification and detection, winning the ImageNet and COCO competitions. This notebook is an implementation of the Residual Network (**ResNet** for short) in PyTorch based on this paper.

This notebook is prepared for students who have participated in the AML course (or a fairly close course). It supposes a basic knowledge of Deep Learning and Convolutional Neural Networks, which have been introduced in the previous courses ([DL](https://github.com/SupaeroDataScience/deep-learning/tree/main/deep), [CNN](https://github.com/fchouteau/isae-practical-deep-learning)), you can refer to them if needed.

**Table of contents:**
0. [Preparation](#sec0)
1. [Problem introduction](#sec1)
2. [Construction of ResNet](#sec2)
    1. [Residual Learning](#sec2-1)
    2. [Identity Mapping by Shortcuts](#sec2-2)
    3. [Network Architectures](#sec2-3)
3. [Experiments](#sec3)
    1. [ImageNet Classification](#sec3-1)
    2. [CIFAR-10 and Analysis](#sec3-2)
4. [Conclusion](#sec4)

# <a id="sec0"></a>0. Preparation

In this notebook, we'll be using `torch` and `torchvision`, which we have already used in previous AML courses. Run the following code blocks to install the necessary packages and verify that everything is working by importing everything. 

Please refer to the [PyTorch](https://pytorch.org/get-started/locally/) website for installation instructions if necessary. We'll also be using packages `sklearn`, `numpy`, and `matplotlib`. 

Note that this notebook is fairly compute intensive and might be better [run in Google Colab].

In [1]:
# !pip install torch torchvision

In [29]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data.dataloader as Data
from torch.autograd import Variable
import torchvision
from torchvision import datasets, models, transforms
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from PIL import Image

# <a id="sec1"></a>1. Problem introduction

From experience, the depth of the network is crucial to the performance of the model. When the number of network layers is increased, the network can extract more complex feature patterns, so theoretically better results can be achieved when the model is deeper. 

But the experiment found that the deep network has a *degradation* problem: with the network depth increasing, accuracy gets saturated (which might be unsurprising) and then degrades rapidly. This phenomenon can be seen directly in Figure 1 which shows the training error (left) and test error (right) on CIFAR-10 with 20-layer and 56-layer "plain" networks. The deeper network has higher training error, and thus test error. This is not caused by overfitting, because the training error of the 56-layer network is also high.

<img src="img/degradation.JPG" width="50%"></img>

<center><font size=1.5><br>Figure 1. Training error (left) and test error (right) on CIFAR-10 with 20-layer and 56-layer "plain" networks.<br>
    The deeper network has higher training error, and thus test error.</font></center>

<div class="alert alert-warning">

**Think about this question:**<br>
How to effectively solve the "degradation" problem caused by the increase in network depth?
    
</div>

<div class="alert alert-danger"><a href="#answer1" data-toggle="collapse"><b>Ready to see the answer? (click to expand)</b></a><br>
<div id="answer1" class="collapse">

The problem of degradation is mainly due to the increase in network depth. During model training, the gradient cannot be effectively transmitted to the shallow network, resulting in [vanishing/exploding gradients](). **Batch Normalization** (BN) changes the data distribution by normalizing the output data, which is a forward process to solve the vanishing/exploding gradients problem. The residual network (ResNet) directly connects the shallow network and the deep network by adding **shortcut connection** (Identity Map), so that the gradient can be well transmitted to the shallow layer.
    
</div>
</div>

# Plain Network

In [2]:
class PlainBlock(nn.Module):
    """
    A basic building block for Plain Network
    
    Parameters：
        - in_channel: Number of input channel
        - out_channel: Number of output channel
        - stride: Number of stride 
        - downsample: "None" for identity downsample, otherwise for a real downsample
    
    """
    
    expansion = 1    # Record whether the number of convolution kernels in each layer has changed
 
    def __init__(self, in_channel, out_channel, stride=1, downsample=None):
        super(PlainBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=in_channel, out_channels=out_channel,
                               kernel_size=3, stride=stride, padding=1, bias=False)  # 有无bias对bn没多大影响
        self.bn1 = nn.BatchNorm2d(out_channel)
        self.relu = nn.ReLU()
 
        self.conv2 = nn.Conv2d(in_channels=out_channel, out_channels=out_channel,
                               kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channel)
 
        self.downsample = downsample
 
    def forward(self, x):
        identity = x      # Record the output of the last residual block
        
        if self.downsample is not None:  # Determine if need to downsample for dimension matching
            identity = self.downsample(x)
 
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
 
        out = self.conv2(out)
        out = self.bn2(out)
 
        out = self.relu(out)
 
        return out

# <a id="sec2"></a>2. Construction of ResNet

### <a id="sec2-1"></a>2.1. Residual Learning

In response to the "degradation" problem, the author Dr. He proposed a **deep residual learning** framework, which uses a multi-layer network to fit a residual mapping.

Formally, denoting the desired underlying mapping as $H(x)$, we let the stacked nonlinear layers fit another mapping:

$$F(x) := H(x)−x$$ 

The original mapping is recast into:

$$F(x)+x$$. 

We hypothesize that it is easier to optimize the residual mapping than to optimize the original, unreferenced mapping. To the extreme, if an identity mapping were optimal, it would be easier to push the residual to zero than to fit an identity mapping by a stack of nonlinear layers.

The formulation of $F(x)+x$ can be realized by feedforward neural networks with "**[shortcut connections]()**". Shortcut connections are those skipping one or more layers. For the case in paper, the shortcut connections simply perform identity mapping, and their outputs are added to the outputs of the stacked layers. 

Figure 2 shows a building block of residual learning in the deep residual network:

<img src="img/2-layer building block.JPG" width="340px">

<center><font size=1.5><br>Figure 2. Residual learning: a building block.</font></center>

Identity shortcut connections add neither extra parameter nor computational complexity. The entire network can still be trained end-to-end by SGD with backpropagation, and can be easily implemented using common libraries (e.g., [Caffe]) without modifying the solvers.

<div class="alert alert-success">

**In brief:**<br>
- If identity mappings are added, a deeper network will not perform worse than a shallow network.
- It is difficult to learn identify mapings in a network structure composed of multiple non-linear layers.
- If identity mapings is the optimal link method, then the weight parameters of $F(x)$ will tend to $0$.
- If the optimal mapping is close to identity mappings, it is much easier to find the $F(x)$ corresponding to the identity mappings (initial parameters near 0) during optimization than to approximately fit a completely new function.
    
</div>

In [12]:
class BasicBlock(nn.Module):
    
    """
    A basic building block for 18/34-layer ResNet.
    
    Parameters：
        - in_channel: Number of input channel
        - out_channel: Number of output channel
        - stride: Number of stride 
        - downsample: "None" for identity downsample, otherwise for a real downsample
    
    """
    
    expansion = 1    # Record whether the number of convolution kernels in each layer has changed
 
    def __init__(self, in_channel, out_channel, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=in_channel, out_channels=out_channel,
                               kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channel)
        self.relu = nn.ReLU(inplace=True)
 
        self.conv2 = nn.Conv2d(in_channels=out_channel, out_channels=out_channel,
                               kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channel)
 
        self.downsample = downsample
 
    def forward(self, x):
        residual = x      # Record the output of the last residual block
 
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
 
        out = self.conv2(out)
        out = self.bn2(out)
        
        if self.downsample is not None:   # Determine if need to downsample for dimension matching
            residual = self.downsample(x)
 
        out += residual
        out = self.relu(out)
 
        return out

### <a id="sec2-2"></a>2.2. Identity Mapping by Shortcuts

Two connection methods are proposed for shortcut connection.

**Method 1:**

$$y = F(x,\{W_{i}\}) + x$$

Where:

- $x$ represents the input vector of the building block for the layers considered.
- $y$ represents the output vector of the building block for the layers considered.
- $F(x,\{W_{i}\})$ represents the residual mapping to be learned, which is the superposition of multiple nonlinear convolutional layers.<br>
  For the example in figure above that has two layers, $F = W_2 \sigma(W_1 x)$ in which $\sigma$ represents the nonlinear activation function ReLU, and the biases are omitted for simplifying notations.
- $F+x$ means shortcut connection, which corresponds to the addition of each pixel.

<div class="alert alert-success">

**Note:** This network structure introduce neither extra parameter nor computation complexity. This is not only attractive in practice but also important in the comparisons between plain and residual networks. We can fairly compare plain/residual networks that simultaneously have the same number of parameters, depth, width, and computational cost (except for the negligible element-wise addition).
    
<div>

**Method 2:**

In method 1, the dimensions of $x$ and $F$ must be equal. If this is not the case (e.g., when changing the input/output channels), we can perform a linear projection $W_s$ by the shortcut connections to match the dimensions:

$$y = F(x,\{W_{i}\}) + W_{s}x$$

Where:

- $x$ represents the input vector of the building block for the layers considered.
- $y$ represents the output vector of the building block for the layers considered.
- $F(x,\{W_{i}\})$ represents the residual mapping to be learned, which is the superposition of multiple nonlinear convolutional layers.<br>
  For the example in figure above that has two layers, $F = W_2 \sigma(W_1 x)$ in which $\sigma$ represents the nonlinear activation function ReLU, and the biases are omitted for simplifying notations.
- $F+x$ means shortcut connection, which corresponds to the addition of each pixel.

<div class="alert alert-success">
    
**Note:** The identity mapping is sufficient for addressing the degradation problem and is economical, thus $W_s$ will be only used when matching dimensions.
    
<div>

It is also mentioned in paper that, for $F(x,\{W_{i}\})$, it should not be limited to the two-layer convolution connection mentioned above, it can be more diverse, such as the three-layer building block on the right of Figure 3. One such small unit is called a *block*. When building a deep network structure, the author calls the second structure *bottleneck* building block.

<img src="img/3-layer building block.JPG">

<center><font size=1.5><br>Figure 3. Two different building blocks for residual learning.<br> 
    Left: a building block (on 56 $\times$ 56 feature maps) as in Figure 4 for ResNet34.<br>
    Right: a "bottleneck" building block for ResNet-50/101/152.</font></center>

In [14]:
class Bottleneck(nn.Module):
    
    """
    A "bottleneck" building block for 50/101/152-layer ResNet.
    
    Parameters：
        - in_channel: Number of input channel
        - out_channel: Number of output channel
        - stride: Number of stride 
        - downsample: "None" for identity downsample, otherwise for a real downsample
    
    """
    
    expansion = 4       # The number of convolution kernels in the third layer (256, 512, 1024, 2048) 
                        # is 4 times the number of convolution kernels in the first or second layer (64, 128, 256, 512)
 
    def __init__(self, in_channel, out_channel, stride=1, downsample=None):
        super(Bottleneck, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=in_channel, out_channels=out_channel,
                               kernel_size=1, stride=1, bias=False)  # Squeeze channels for dimensionality reduce
        self.bn1 = nn.BatchNorm2d(out_channel)
        self.relu = nn.ReLU(inplace=True)
 
        self.conv2 = nn.Conv2d(in_channels=out_channel, out_channels=out_channel,
                               kernel_size=3, stride=stride, bias=False, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channel)
        self.relu = nn.ReLU(inplace=True)
 
        self.conv3 = nn.Conv2d(in_channels=out_channel, out_channels=out_channel*self.expansion,
                               kernel_size=1, stride=1, bias=False)  # Unsqueeze channels for dimensionality increase
        self.bn3 = nn.BatchNorm2d(out_channel*self.expansion)
 
        self.downsample = downsample
 
    def forward(self, x):
        residual = x
 
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
 
        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)
 
        out = self.conv3(out)
        out = self.bn3(out)
        
        if self.downsample is not None:
            residual = self.downsample(x)
 
        out += residual
        out = self.relu(out)
 
        return out

### <a id="sec2-3"></a>2.3. Network Architectures

The subsequent implementation part is mainly to compare the two network structures of **plain nets** and **residual nets**, so this part focuses on the description of these two network structures.

<img src="img/plain-res nets.jpg" width="55%">

<center><font size=1.5><br>Figure 4. Example network architectures for ImageNet.<br>
    Left: the [VGG-19] model(19.6 billion FLOPs) as a reference.<br> 
    Middle: a plain network with 34 parameter layers (3.6 billion FLOPs).<br>
    Right: a residual network with 34 parameter layers (3.6 billionFLOPs).<br>
    The dotted shortcuts increase dimensions.</font></center>

**Plain Network**

The plain baselines (Figure 4, middle) are mainly inspired by the philosophy of [VGG nets] (Figure 4, left). The convolutional layers mostly have $3 \times 3$ filters and follow two simple design rules: (i) for the same output feature map size, the layers have the same number of filters; and (ii) if the feature map size is halved, the number of filters is doubled so as to preserve the time complexity per layer. We perform downsampling directly by convolutional layers that have a stride of 2. The network ends with a global average pooling layer and a 1000-way fully-connected layer with softmax. The total number of weighted layers is 34 (Figure 4, middle).

**Residual Network**

Based on the above plain network, we insert shortcut connections (Figure 4, right) which turn the network into its counterpart residual version. The identity shortcuts (method 1) can be directly used when the input and output are of the same dimensions (solid line shortcuts in Figure 4). When the dimensions increase (dotted line shortcuts in Figure 4), we consider two options: (A) The shortcut still performs identity mapping, with extra zero entries padded for increasing dimensions. This option introduces no extra parameter; (B) The projection shortcut in method 2 is used to match dimensions (done by 1 $\times$ 1 convolutions). For both options, when the shortcuts go across feature maps of two sizes, they are performed with a stride of 2.

In [45]:
class ResNet(nn.Module):
    """
    Implementation of ResNet architecture.
    
    Parameters：
        - block: "BasicBlock" for 18/34-layer ResNet, "Bottleneck" for 50/101/152-layer ResNet
        - blocks_num: The number of residual layers, for example, [3,4,6,3] for the 34-layer ResNet
        - include_top: Facilitate the construction of more complex networks on the basis of resnet in the future

    """
 
    def __init__(self, block, blocks_num, num_classes=10):
        super(ResNet, self).__init__()
        self.in_channel = 64    # The number of output channels in the previous layer, also the number of input channels in this layer
 
        #  part 1: conv1 + maxpooling
        self.conv1 = nn.Conv2d(1, self.in_channel, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(self.in_channel)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
 
        #  part 2: conv2,3,4,5
        self.layer1 = self._make_layer(block, 64, blocks_num[0], stride=1)
        self.layer2 = self._make_layer(block, 128, blocks_num[1], stride=2)
        self.layer3 = self._make_layer(block, 256, blocks_num[2], stride=2)
        self.layer4 = self._make_layer(block, 512, blocks_num[3], stride=2)
 
        #  part 3: avgpooling + fully connected layer
        self.avgpool = nn.AvgPool2d(7)
        self.fc = nn.Linear(256 * block.expansion, num_classes)
 
        #  Initialization of the convolutional layer
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
 
    def _make_layer(self, block, channel, block_num, stride=1):
        downsample = None
        if stride != 1 or self.in_channel != channel * block.expansion:
            #  The dotted line in the ResNet architecture
            downsample = nn.Sequential(
                nn.Conv2d(self.in_channel, channel * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(channel * block.expansion))
 
        layers = []
        layers.append(block(self.in_channel, channel, downsample=downsample, stride=stride))
        self.in_channel = channel * block.expansion
 
        for _ in range(1, block_num):
            layers.append(block(self.in_channel, channel))
 
        return nn.Sequential(*layers)   # Convert list into non-keyword parameters
 
    def forward(self, x):
 
        # part 1
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        # x = self.maxpool(x)
 
        # part 2
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        # x = self.layer4(x)
 
        # part 3
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
 
        return x

In [16]:
__all__ = ['ResNet', 'resnet18', 'resnet34', 'resnet50', 'resnet101', 'resnet152']

Now we can construct a 34-layer ResNet with "BasicBlock" and a 101-layer ResNet with "Bottleneck".

In [46]:
def resnet18(pretrained=False, num_classes=10):
    """Constructs a ResNet-18 model.
    Args:
        pretrained (boolean): If True, returns a model pre-trained on ImageNet
    """
    return ResNet(BasicBlock, [2, 2, 2, 2], num_classes=num_classes)

In [6]:
def plainnet34(num_classes=1000, include_top=True):
    return ResNet(PlainBlock, [3, 4, 6, 3], num_classes=num_classes, include_top=include_top)

In [7]:
def resnet34(num_classes=1000, include_top=True):
    return ResNet(BasicBlock, [3, 4, 6, 3], num_classes=num_classes, include_top=include_top)

In [8]:
def resnet101(num_classes=1000, include_top=True):
    return ResNet(Bottleneck, [3, 4, 23, 3], num_classes=num_classes, include_top=include_top)

# <a id="sec3"></a>3. Experiments

In [25]:
import argparse

parser = argparse.ArgumentParser()
parser.add_argument("--model", type=str, default='resnet18', help="model")
parser.add_argument("--patience", type=int, default=3, help="early stopping patience")
parser.add_argument("--batch_size", type=int, default=256, help="batch size")
parser.add_argument("--nepochs", type=int, default=200, help="max epochs")
parser.add_argument("--nworkers", type=int, default=4, help="number of workers")
parser.add_argument("--seed", type=int, default=1, help="random seed")
# parser.add_argument("--data", type=str, default='MNIST', help="MNIST, or FashionMNIST")
args = parser.parse_args(args=[])

In [26]:
# Set up the device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('Training on {}'.format(device))

Training on cpu


In [27]:
# Set seeds. If using numpy this must be seeded too.
torch.manual_seed(1)
if device== 'cuda:0':
    torch.cuda.manual_seed(1)

## Fashion-MNIST dataset

In [ ]:
labels_text = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat", "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

In [31]:
# Define transforms.
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.1307,), (0.3081,))])

In [39]:
full_trainset = torchvision.datasets.FashionMNIST(root='../data', train=True, download=True, transform=transform)
trainset, full_validset = torch.utils.data.random_split(full_trainset, (10000, 50000))
validset, _ = torch.utils.data.random_split(full_validset, (1000, 49000))

trainloader = torch.utils.data.DataLoader(trainset, batch_size=512, shuffle=True, num_workers=2)
validloader = torch.utils.data.DataLoader(validset, batch_size=512, shuffle=True, num_workers=2)

In [5]:
# get the images and labels with parameter batch_size=4
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True, num_workers=2)
dataiter = iter(trainloader)
images, labels = dataiter.next()

plt.figure(figsize=(4,4))
for i in range(len(images)):
    l = labels[i].numpy()
    plt.subplot(2, 2, i+1)
    plt.title('%d: %s' % (l, labels_text[l]))
    plt.imshow(images[i].numpy()[0], cmap='Greys')
    plt.axis('off')

In [ ]:
criterion = nn.CrossEntropyLoss()

def validation(net):
    valid_loss = 0
    with torch.no_grad():
        for data in validloader:
            images, labels = data
            outputs = net(images)
            loss = criterion(outputs, labels)
            valid_loss += loss.item()
    return valid_loss

def train(net):
    optimizer = torch.optim.SGD(net.parameters(), lr=0.1, momentum=0.9)
    train_history = []
    valid_history = []
    for epoch in range(30):
        train_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            # zero the parameter gradients
            optimizer.zero_grad()
            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        valid_loss = validation(net)
        train_history.append(train_loss)
        valid_history.append(valid_loss)
        print('Epoch %02d: train loss %0.5f, validation loss %0.5f' % (epoch, train_loss, valid_loss))
    return train_history, valid_history

In [ ]:
net = resnet18(pretrained=False, num_classes=10)
# net = net.to(device)
train_history, valid_history = train(net)

In [34]:
def run_model(net, loader, criterion, optimizer, train = True):
    running_loss = 0
    running_accuracy = 0
    running_history = []

    # Set mode
    if train:
        net.train()
    else:
        net.eval()


    for i, data in enumerate(loader):
        per_run_loss = 0.0

        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        
        # Pass to gpu or cpu
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        with torch.set_grad_enabled(train):
            output = net(inputs)
            _, pred = torch.max(output, 1)
            loss = criterion(output, labels)

        # If on train backpropagate
        if train:
            loss.backward()
            optimizer.step()

        # Calculate stats
        per_run_loss += loss.item()
        running_loss += loss.item()
        running_accuracy += torch.sum(pred == labels.detach())
        running_history.append(running_loss)
        if train:
            print('Epoch %02d: train loss %0.5f' % (epoch, per_run_loss))
        else:
            print('Epoch %02d: validation loss %0.5f' % (epoch, per_run_loss))
        
    return running_loss / len(loader), running_accuracy.double() / len(loader.dataset), running_history

In [47]:
# Init network, criterion and early stopping
net = resnet18(pretrained=False, num_classes=10)
# net = net.to(device)
criterion = torch.nn.CrossEntropyLoss()

In [49]:
# Define optimizer
optimizer = torch.optim.Adam(net.parameters())

In [1]:
# Train the network
import time
# import utils

patience = 3
best_loss = 1e4
    
for epoch in range(30):
    start = time.time()
    train_loss, train_acc, train_history = run_model(net, trainloader,
                                      criterion, optimizer)
    val_loss, val_acc, valid_history = run_model(net, validloader,
                                  criterion, optimizer, False)
    end = time.time()

    # print stats
    stats = """Epoch: {}\t train loss: {:.3f}, train acc: {:.3f}\t
            val loss: {:.3f}, val acc: {:.3f}\t
            time: {:.1f}s""".format(epoch, train_loss, train_acc, val_loss,
                                        val_acc, end - start)
    print(stats)

#     # early stopping and save best model
#     if val_loss < best_loss:
#         best_loss = val_loss
#         patience = patience
#         utils.save_model({
#             'arch': net,
#             'state_dict': net.state_dict()
#         }, 'saved-models/{}-run-{}.pth.tar'.format(net, run))
#     else:
#         patience -= 1
#         if patience == 0:
#             print('Run out of patience!')
#             break

**由于jupyter notebook运行不起来，因此我转到colab中运行了一下。<br>用CPU运行，1个epoch的大约需要5min；用GPU运行，1个epoch大约需要6s。**

**Epoch: 1**	 train loss: 0.460, train acc: 0.833	
            val loss: 0.706, val acc: 0.749	
            time: 283.9s

In [ ]:
def get_valid_predictions(net):
    # validloader = torch.utils.data.DataLoader(validset, batch_size=4, shuffle=False)
    all_labels = np.array([])
    predictions = np.array([])
    with torch.no_grad():
        for data in validloader:
            images, labels = data
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            all_labels = np.append(all_labels, labels.numpy())
            predictions = np.append(predictions, predicted.numpy())
    return all_labels, predictions

In [ ]:
y_valid, predictions = get_valid_predictions(net)

from sklearn.metrics import accuracy_score, classification_report

print('Accuracy: ', accuracy_score(predictions, y_valid))
print(classification_report(predictions, y_valid, target_names=labels_text))

**以下是colab中的运行结果：**

Accuracy:  0.749<br>
                
                precision    recall  f1-score   support

    T-shirt/top       0.84      0.60      0.70       129
     Trouser       0.95      0.99      0.97        88
    Pullover       0.85      0.68      0.76       141
       Dress       0.56      0.97      0.71        64
        Coat       0.57      0.65      0.61        89
      Sandal       0.40      1.00      0.57        37
       Shirt       0.46      0.41      0.43        91
     Sneaker       0.96      0.66      0.78       157
         Bag       0.90      0.96      0.93        92
    Ankle boot       0.93      0.92      0.92       112

    accuracy                           0.75      1000
    macro avg       0.74      0.78      0.74      1000
    weighted avg       0.79      0.75      0.75      1000

In [3]:
def plot_train_val(train, valid):
    fig, ax1 = plt.subplots()
    color = 'tab:red'
    ax1.set_ylabel('Training', color=color)
    ax1.plot(train, color=color)
    ax2 = ax1.twinx()
    color = 'tab:blue'
    ax2.set_ylabel('Validation', color=color)
    ax2.plot(valid, color=color)
    fig.tight_layout()
    
plot_train_val(train_history, valid_history)

下面对测试集进行测试：

In [ ]:
testset = torchvision.datasets.FashionMNIST(root='../data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(trainset, batch_size=512, shuffle=False, num_workers=2)

In [ ]:
def run_test(net):
    # Model = model.__dict__['resnet18']().to(device)
    # Model.load_state_dict(torch.load(path)['state_dict'])
    with torch.no_grad():  ## 禁止使用反传
        pred = []
        all_label = []
        for batch_idx, (data, label) in enumerate(testloader):
            batch_x, batch_y = data.to(device), label.to(device)
            batch_x, batch_y = Variable(batch_x), Variable(batch_y)
            output = net(batch_x)
            
            pred2 = output.max(1, keepdim=True)[1]
            pred2 = pred2.cpu().numpy()  
            for ii in range(len(pred2)):
                pred.append((pred2[ii])[0])
            all_label = np.append(all_label, batch_y.numpy())

    return pred, all_label

In [ ]:
pred, y_test = run_test(net)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

print('Accuracy: ', accuracy_score(pred, y_test))
print(classification_report(pred, y_test, target_names=labels_text))

**colab中的运行epoch=1的结果：**

Accuracy:  0.779<br>
              
              precision    recall  f1-score   support

    T-shirt/top       0.89      0.65      0.75      1401
     Trouser       0.96      0.99      0.98       982
    Pullover       0.82      0.69      0.75      1186
       Dress       0.52      0.98      0.68       547
        Coat       0.69      0.65      0.67      1059
      Sandal       0.51      1.00      0.67       492
       Shirt       0.57      0.58      0.57       959
     Sneaker       0.96      0.69      0.80      1414
         Bag       0.91      0.95      0.93       972
    Ankle boot       0.95      0.93      0.94       988

    accuracy                           0.78     10000
    macro avg       0.78      0.81      0.77     10000
    weighted avg       0.82      0.78      0.78     10000

### MNIST dataset

In [61]:
train_data = torchvision.datasets.MNIST(
    './mnist', train=True, transform=torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    ]), download=True
)

train_data.data = train_data.data[:10000]
train_data.targets = train_data.targets[:10000]

test_data = torchvision.datasets.MNIST(
    './mnist', train=False, transform=torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    ]), download=True
)

print("train_data:", train_data.train_data.size())
print("train_labels:", train_data.train_labels.size())
print("test_data:", test_data.test_data.size())

Extracting ./mnist\MNIST\raw\train-images-idx3-ubyte.gz to ./mnist\MNIST\raw



Extracting ./mnist\MNIST\raw\train-labels-idx1-ubyte.gz to ./mnist\MNIST\raw


Extracting ./mnist\MNIST\raw\t10k-images-idx3-ubyte.gz to ./mnist\MNIST\raw


Extracting ./mnist\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./mnist\MNIST\raw
Processing...
Done!
train_data: torch.Size([10000, 28, 28])
train_labels: torch.Size([10000])
test_data: torch.Size([10000, 28, 28])


D:\Anaconda3\lib\site-packages\torchvision\datasets\mnist.py:55: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")
D:\Anaconda3\lib\site-packages\torchvision\datasets\mnist.py:45: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")
D:\Anaconda3\lib\site-packages\torchvision\datasets\mnist.py:60: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")


In [62]:
train_loader = Data.DataLoader(dataset=train_data, batch_size=32, shuffle=True)
test_loader = Data.DataLoader(dataset=test_data, batch_size=32)

model = ResNet18()
if if_use_gpu:
    model = model.cuda()

print(model)

ResNet(
  (conv1): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (layer1): Sequential(
    (0): ResidualBlock(
      (left): Sequential(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
        (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (right): Sequential()
    )
    (1): ResidualBlock(
      (left): Sequential(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
        (3): Con

In [63]:
optimizer = torch.optim.Adam(model.parameters())
loss_func = torch.nn.CrossEntropyLoss()

for epoch in range(1):
    print('epoch {}'.format(epoch + 1))
    for i, data in enumerate(train_loader, 0):
        # get the inputs
        inputs, labels = data
        batch_x, batch_y = Variable(inputs), Variable(labels)
        if if_use_gpu:
            batch_x = batch_x.cuda()
            batch_y = batch_y.cuda()
        out = model(batch_x)
        batch_y = batch_y.long()
        loss = loss_func(out, batch_y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # 返回每行元素最大值
        pred = torch.max(out, 1)[1]
        train_correct = (pred == batch_y).sum()
        train_correct = train_correct.item()
        train_loss = loss.item()
        print('batch:{},Train Loss: {:.6f}, Acc: {:.6f}'.format(i+1,train_loss , train_correct /32))

epoch 1
batch:1,Train Loss: 2.374657, Acc: 0.062500
batch:2,Train Loss: 6.123889, Acc: 0.156250
batch:3,Train Loss: 19.508154, Acc: 0.125000
batch:4,Train Loss: 13.292258, Acc: 0.218750
batch:5,Train Loss: 4.874920, Acc: 0.125000
batch:6,Train Loss: 6.002936, Acc: 0.062500
batch:7,Train Loss: 15.337273, Acc: 0.093750
batch:8,Train Loss: 5.083591, Acc: 0.156250
batch:9,Train Loss: 7.783920, Acc: 0.093750
batch:10,Train Loss: 9.707572, Acc: 0.125000
batch:11,Train Loss: 6.786269, Acc: 0.156250
batch:12,Train Loss: 6.968081, Acc: 0.312500
batch:13,Train Loss: 5.258401, Acc: 0.125000
batch:14,Train Loss: 7.909595, Acc: 0.125000
batch:15,Train Loss: 3.568305, Acc: 0.250000
batch:16,Train Loss: 3.798988, Acc: 0.218750
batch:17,Train Loss: 5.174158, Acc: 0.093750
batch:18,Train Loss: 2.668407, Acc: 0.437500
batch:19,Train Loss: 2.718485, Acc: 0.437500
batch:20,Train Loss: 3.481237, Acc: 0.218750
batch:21,Train Loss: 2.669489, Acc: 0.312500
batch:22,Train Loss: 3.429018, Acc: 0.156250
batch:23

batch:182,Train Loss: 0.412537, Acc: 0.843750
batch:183,Train Loss: 0.203275, Acc: 0.968750
batch:184,Train Loss: 0.417183, Acc: 0.875000
batch:185,Train Loss: 0.030099, Acc: 1.000000
batch:186,Train Loss: 0.200837, Acc: 0.937500
batch:187,Train Loss: 0.127633, Acc: 0.937500
batch:188,Train Loss: 0.048370, Acc: 0.968750
batch:189,Train Loss: 0.322275, Acc: 0.937500
batch:190,Train Loss: 0.079107, Acc: 0.968750
batch:191,Train Loss: 0.252487, Acc: 0.906250
batch:192,Train Loss: 0.180565, Acc: 0.937500
batch:193,Train Loss: 0.293501, Acc: 0.906250
batch:194,Train Loss: 0.387668, Acc: 0.937500
batch:195,Train Loss: 0.668451, Acc: 0.843750
batch:196,Train Loss: 0.509568, Acc: 0.812500
batch:197,Train Loss: 0.283848, Acc: 0.906250
batch:198,Train Loss: 0.103716, Acc: 0.968750
batch:199,Train Loss: 0.071170, Acc: 0.968750
batch:200,Train Loss: 0.123919, Acc: 0.968750
batch:201,Train Loss: 0.198210, Acc: 0.937500
batch:202,Train Loss: 0.197717, Acc: 0.875000
batch:203,Train Loss: 0.278053, Ac

KeyboardInterrupt: 

In [69]:
# Evaluation--------------------------------
model.eval()
eval_loss = 0.
eval_acc = 0.

for batch_x, batch_y in test_loader:
    batch_x, batch_y = Variable(batch_x, requires_grad=False), Variable(batch_y, requires_grad=False)
    if if_use_gpu:
        batch_x = batch_x.cuda()
        batch_y = batch_y.cuda()
    out = model(batch_x)
    loss = loss_func(out, batch_y)
    eval_loss += loss.item()
    pred = torch.max(out, 1)[1]
    num_correct = (pred == batch_y).sum()
    eval_acc += num_correct.item()
    
print('Test Loss: {:.6f}, Acc: {:.6f}'.format(eval_loss / (len(test_data)), eval_acc / (len(test_data))))

RuntimeError: [enforce fail at ..\c10\core\CPUAllocator.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 57802752 bytes. Buy new RAM!


### Train with Early stopping (draft...)

In [28]:
class EarlyStopping:
    
    def __init__(self, patience=5, delta=0):
        self.patience = patience
        self.counter = 0
        self.best_score = None
        self.delta = delta
        self.early_stop = False

    def step(self, val_loss):
        score = -val_loss
        if self.best_score is None:
            self.best_score = score
        elif score < self.best_score + self.delta:
            self.counter += 1
            print('EarlyStopping counter: %d / %d' % (self.counter, self.patience))
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.counter = 0

In [29]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=512, shuffle=True, num_workers=2)
validloader = torch.utils.data.DataLoader(validset, batch_size=512, shuffle=True, num_workers=2)
criterion = nn.CrossEntropyLoss()

def validation(net):
    valid_loss = 0
    with torch.no_grad():
        for data in validloader:
            images, labels = data
            outputs = net(images)
            loss = criterion(outputs, labels)
            valid_loss += loss.item()
    return valid_loss

def train2(net, earlystopping=True):
    optimizer = torch.optim.SGD(net.parameters(), lr=0.1, momentum=0.9)
    train_history = []
    valid_history = []
    estop = EarlyStopping(patience=2)
    for epoch in range(30):
        train_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            # zero the parameter gradients
            optimizer.zero_grad()
            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        valid_loss = validation(net)
        train_history.append(train_loss)
        valid_history.append(valid_loss)
        print('Epoch %02d: train loss %0.5f, validation loss %0.5f' % (epoch, train_loss, valid_loss))
        estop.step(valid_loss)
        if earlystopping and estop.early_stop:
            break
    return train_history, valid_history